<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/8_Create_Generic_Sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set-up
import pandas as pd
import numpy as np
from google.colab import files

pd.set_option('display.max_columns', None)

def dlcsv(df,name):
  df.to_csv('{}.csv'.format(name), index=False)
  files.download('{}.csv'.format(name))

In [ ]:
#combine weather and crime for each year

data= []

for year in range(2015,2020):
  weather = pd.read_csv('https://raw.githubusercontent.com/MattIzon/16010269_DataAnalytics/main/data/w_{}.csv'.format(year)).set_index('date')
  crime = pd.read_csv('https://raw.githubusercontent.com/MattIzon/16010269_DataAnalytics/main/data/c_day_{}.csv'.format(year)).set_index('date')
  crime.drop('Unnamed: 0', axis=1, inplace=True)
  data.append(crime.join(weather).reset_index())

In [ ]:
# Remove outliers (more than 2 standard deviations from mean)
adjusted = []

for year in range(5):
  df = data[year].copy()
  upper = df.crime_count.mean() + (2 * df.crime_count.std())
  lower = df.crime_count.mean() - (2 * df.crime_count.std())

  df.drop(df[df.crime_count > upper].index, inplace=True)
  df.drop(df[df.crime_count < lower].index, inplace=True)

  adjusted.append(df)

In [ ]:
# Combine all years into single dataset
combined = pd.concat(adjusted).reset_index()

In [ ]:
# Re-allocate day_of_week categories to produce a linear relationship
combined.groupby('day_of_week').mean()['crime_count'].sort_values() # new order

cda = combined.copy() # cda ~ combined_day-of-week_adjusted
label_conversions = {1:3,
                     2:7,
                     3:5,
                     4:4,
                     5:1,
                     6:2,
                     7:6}
cda['day_of_week'].replace(label_conversions, inplace=True)

In [ ]:
rand_combined = combined.iloc[np.random.permutation(len(combined))]

In [ ]:
test_set = rand_combined.iloc[:5]
dlcsv(test_set, 'test_data')

train_size = int((len(rand_combined)-5)*0.8)
train_set = rand_combined.iloc[5:train_size]
evaluate_set = rand_combined.iloc[train_size:]
dlcsv(train_set, 'train_data')
dlcsv(evaluate_set, 'eval_data')